In [1]:
from __future__ import print_function 

def hangman(secret_word, guesser, max_mistakes=11):
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = list()
    
    mistakes = 0
    while mistakes < max_mistakes:
        guess = str(guesser(mask, guessed))
        if guess in guessed:
            mistakes += 1
        else:
            try:
                guessed.append(guess)
            except:
                print(guessed,guess)
            if guess in secret_word:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
            else:
                mistakes += 1
                
        if '_' not in mask:
            return mistakes,guessed        
    
    return mistakes,guessed

# loading data:

In [20]:
ftrain = open('train google.txt',encoding="utf8").read().splitlines()
ftest = open('test.txt',encoding="utf8").read().splitlines()

test = set()
train=set()
for i,item in enumerate(ftest):
    test.add(item)
    if len(test)==1000:
        break
for i,item in enumerate(ftrain):
    train.add(item)

len(train), len(test)

(173528, 1000)

# unigram with position bidirectionality:

In [3]:
from collections import defaultdict
frequni = defaultdict(dict)
posCountDict = dict()
frequniRev = defaultdict(dict)
posCountDictRev = dict()


for item in train:
    for pos,chara in enumerate(item):
        revIndex = len(item)-1-pos

        try:
            frequni[pos][chara] += 1
        except KeyError:
            frequni[pos][chara] = 1
            
        try:
            frequniRev[revIndex][chara] += 1
        except KeyError:
            frequniRev[revIndex][chara] = 1
            
            
        try:
            posCountDict[pos] += 1
        except KeyError:
            posCountDict[pos] = 1
            
        try:
            posCountDictRev[revIndex] += 1
        except KeyError:
            posCountDictRev[revIndex] = 1


freqProb = defaultdict(dict)
freqProbRev = defaultdict(dict)

for item in frequni.keys():
    if posCountDict[item] > 50:
        for stuff in frequni[item].keys():
            freqProb[item][stuff] = frequni[item][stuff]*1.0/posCountDict[item]
            
for item in frequniRev.keys():
    if posCountDictRev[item] > 50:
        for stuff in frequniRev[item].keys():
            freqProbRev[item][stuff] = frequniRev[item][stuff]*1.0/posCountDictRev[item]

In [4]:
import operator
def unigPosBi(mask,guessed):
    locDict = list()
    for item in freqProb.keys():
        if item <= len(mask):
            sorted_x = sorted(freqProb[item].items(), key=operator.itemgetter(1),reverse=True)
            for stuff in sorted_x:
                if stuff[0] not in guessed:
                    locDict.append((stuff[0],stuff[1],item))
                    break
    for item in freqProbRev.keys():
        if item <= len(mask):
            sorted_x = sorted(freqProbRev[item].items(), key=operator.itemgetter(1),reverse=True)
            for stuff in sorted_x:
                if stuff[0] not in guessed:
                    locDict.append((stuff[0],stuff[1],-1*item))
                    break
    loctDict2 = dict()
    for item in locDict:
        try:
            loctDict2[item[0]] += item[1]
        except:
            loctDict2[item[0]] = item[1]
    
    #sorted_x = sorted(locDict, key=lambda x: x[1],reverse=True)
    sorted_x = sorted(loctDict2.items(), key=operator.itemgetter(1),reverse=True)
    for stuff in sorted_x:
        if stuff[0] not in guessed:
            return stuff[0]    

# bidirectonal(position) bigram:

In [5]:
from collections import defaultdict
bigram = defaultdict(dict)
bbigram = defaultdict(dict)
for item in train:
    item2 = '$'+item+'$'
    
    for i,chara in enumerate(item2[:-1]):
        c=1
        while i+c<len(item2)-1:
            try:
                bigram[chara][item2[i+c]] += 1
            except:
                bigram[chara][item2[i+c]] = 1
            c+=1
    for i,chara in enumerate(item2[:-1]):
        c=1
        while i-c>1:
            try:
                bbigram[chara][item2[i-c]] += 1
            except:
                bbigram[chara][item2[i-c]] = 1
            c+=1

In [6]:
outerKeys = dict()
innerKeys=dict()
for item in bigram.keys():
    for stuff in bigram[item]:
        try:
            outerKeys[item] += bigram[item][stuff]
        except:
            outerKeys[item] = bigram[item][stuff]
for item in bbigram.keys():
    for stuff in bbigram[item]:
        try:
            innerKeys[item] += bbigram[item][stuff]
        except:
            innerKeys[item] = bbigram[item][stuff]
            
for item in bigram.keys():
    try:
        del bigram[item]['$']
    except:
        pass
    
    for stuff in bigram[item]:
        bigram[item][stuff] = (1.0*bigram[item][stuff])/outerKeys[item]
        
for item in bigram.keys():
    for i,stuff1 in enumerate(bigram[item]):
        for j,stuff2 in enumerate(bigram[item]):
            if i!=j and stuff1==stuff2:
                bigram[item][stuff1]+=bigram[item][stuff2]
                
for item in bbigram.keys():
    try:
        del bbigram[item]['$']
    except:
        pass
    
    for stuff in bbigram[item]:
        bbigram[item][stuff] = (1.0*bbigram[item][stuff])/innerKeys[item]
        
for item in bbigram.keys():
    for i,stuff1 in enumerate(bbigram[item]):
        for j,stuff2 in enumerate(bbigram[item]):
                if i!=j and stuff1==stuff2:
                    bbigram[item][stuff1]+=bbigram[item][stuff2]
                
for item in bbigram.keys():
    for stuff in bbigram[item]:
        try:
            bigram[item][stuff]+=bbigram[item][stuff]
        except:
            bigram[item][stuff]=bbigram[item][stuff]

def bigramM(mask,guessed):
    currMax = 0.0
    
    currBi = '_'
    if len(mask) == mask.count('_'):
        return unigPosBi(mask,guessed).split('_')[0]
    else:
        for i,item in enumerate(mask):
            try:
                if item != '_' and mask[i+1] == '_':
                    sorted_x = sorted(bigram[item].items(), key=operator.itemgetter(1),reverse=True)
                    for thing in sorted_x:
                        if thing[0] not in guessed: #thing(1)>currmax
                            currMax = thing[1]
                            currBi = thing[0]
                            break
            except IndexError:
                pass

        return currBi

In [21]:
corrs = 0
d1=dict()
for i,item in enumerate(test):
    mist,d1[item] = hangman(item, bigramM, 11)
    if mist < 11:
        corrs += 1
    #d1[item]=guessedLetters
#print (d1)
print (corrs)

459


Accuracy is 45.9%.

# bidirectional(position) n-gram:

In [8]:
from collections import defaultdict

def ngramModel(train,ngt,countsOnly=1):
    ng = ngt - 1
    bigram = defaultdict(dict)
    bbigram = defaultdict(dict)
    outerKeys =dict()
    
    for item in train:
        item2 = '$'+item+'$'
        for i,chara in enumerate(item2):
            c=0
            if i+ng >= len(item2)-1:
                break
            while i+ng+c<len(item2)-1 and c<2:
                try:
                    bigram[item2[i:i+ng]][item2[i+ng+c]] += 1                
                except:
                    try: 
                        bigram[item2[i:i+ng]][item2[i+ng+c]] = 1                    
                    except IndexError:
                        pass
                c+=1
            
    for item in bigram.keys():
        for stuff in bigram[item]:
            try:
                outerKeys[item] += bigram[item][stuff]
            except:
                outerKeys[item] = bigram[item][stuff]
    
    if countsOnly != 1:
        for item in bigram.keys():
            try:
                del bigram[item]['$']
            except:
                pass
            for stuff in bigram[item]:
                bigram[item][stuff] = (1.0*bigram[item][stuff])/outerKeys[item]
        for item in bigram.keys():
            for i,stuff1 in enumerate(bigram[item]):
                for j,stuff2 in enumerate(bigram[item]):
                    if i!=j and stuff1==stuff2:
                        bigram[item][stuff1]+=bigram[item][stuff2]
        
    return bigram,outerKeys

In [9]:
ngModels = dict()
outKeys = dict()

for i in range(2,6):
    ngModels[i],outKeys[i] = ngramModel(train,i)

# Determining the parameters based on probability of n-gram :

In [16]:
def ngraInterpWeights(mask,guessed,paramList=[ 0.25,0.25,0.25,0.25]):
    proList = list()
    scores = defaultdict(dict)
    maski = '$' +  ''.join(mask) + '$'
    if len(mask) == mask.count('_'):
        return unigPosBi(mask,guessed),'nil'
    maskSort = sorted([item[-4:] for item in maski.split('_') if len(item)>=1 and '$'!=item[-1]],key=lambda x: len(x),reverse=True)
    
    for item in maskSort:
        candidates = dict() 
        
        for i in range(-1*(len(item)-1),1):
            
            proAdded = 0
            try:
                pro = sorted(ngModels[len(item[-i:])+1][item[-i:]].items(),key=operator.itemgetter(1),reverse=True)
                for pros in pro:
                    if pros[0] not in guessed:
                        try:
                            candidates[pros[0]] += paramList[-i]* pros[1]*1.0/outKeys[len(item[-i:])+1][item[-i:]]
                        except:
                            candidates[pros[0]] = paramList[-i]* pros[1]*1.0/outKeys[len(item[-i:])+1][item[-i:]]
                        scores[item+'_'+pros[0]][len(item[-i:])+1] =  paramList[-i]* pros[1]*1.0/outKeys[len(item[-i:])+1][item[-i:]]

            except (IndexError,KeyError) as e:
                pass
        finMark = list(sorted(candidates.items(), key=operator.itemgetter(1),reverse=True)[0])
        finMark.append(item)
        proList.append(tuple(finMark))                
    
    if len(proList) == 0:
        return unigPosBi(mask,guessed),'nil'
    
    finVal = sorted(proList, key=lambda x: x[0],reverse=True)[0][0]
    newScores = defaultdict(dict)
    for item in scores.keys():
        if item.split('_')[1]== finVal:
            newScores[item] = scores[item]
    return finVal,newScores


def hangman3(secret_word, guesser, max_mistakes=8, verbose=False,param=20):
    tuples = open('filePar.csv','a')
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess,scores = guesser(mask, guessed,param)
        finScores = defaultdict(dict)
        for i in range(1,6):
            finScores[guess][i] = 0.0
        if scores != 'nil': 
            for item in scores.keys():
                for stuff in scores[item].keys():
                    try:
                        finScores[guess][stuff] += scores[item][stuff]
                    except KeyError:
                        print(item,stuff)

                
        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
                finScores[guess]['correct'] = 1
            else:
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                finScores[guess]['correct'] = 0
        if scores != 'nil':
            print(guess,finScores[guess][2],finScores[guess][3],finScores[guess][4],finScores[guess][5],finScores[guess]['correct'],file=tuples,sep=',')
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    tuples.close()
    return mistakes

# logistic regression:

In [11]:
import pandas
params = pandas.read_csv('filePar.csv',names=['char','bi','tri','quad','five','label'])
params.groupby('label').count()

,char,bi,tri,quad,five
label,,,,,
0,185337,185337,185337,185337,185337
1,181193,181193,181193,181193,181193


In [12]:
from sklearn import linear_model
log_model = linear_model.LogisticRegression(max_iter=50000)
train_features = params[['bi','tri','quad','five']]
test_features = params['label']
log_model.fit(X = train_features ,
              y = test_features)

print(log_model.coef_,log_model.get_params())

[[ 9.03816849  7.08968435  6.59863748  5.82646768]] {'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'max_iter': 50000, 'multi_class': 'ovr', 'n_jobs': 1, 'penalty': 'l2', 'random_state': None, 'solver': 'liblinear', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [13]:
a = log_model.coef_[0]
b = [item/sum(a) for item in a]
#b,b[::-1]

In [22]:
corrs = 0
d2=dict()
totMist=0
for i,item in enumerate(test):
    mist = hangman3(item, ngraInterpWeights, 11,False,b)
    totMist += mist
    if mist < 11:
        corrs += 1
    #print(d2)
print (corrs)

757


Accuracy is 75.7%.